In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import sys

In [42]:
df = pd.read_csv('../data/processed/combined_features.csv')
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 82011 entries, 0 to 82010
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   place_id              82011 non-null  float64
 1   hour                  82011 non-null  int64  
 2   item_count            82011 non-null  float64
 3   order_count           82011 non-null  int64  
 4   total_revenue         82011 non-null  float64
 5   avg_order_value       82011 non-null  float64
 6   avg_items_per_order   82011 non-null  float64
 7   datetime              82011 non-null  str    
 8   day_of_week           82011 non-null  int64  
 9   month                 82011 non-null  int64  
 10  week_of_year          82011 non-null  int64  
 11  type_id               80433 non-null  float64
 12  waiting_time          82009 non-null  float64
 13  rating                82009 non-null  float64
 14  delivery              82009 non-null  float64
 15  accepting_orders      82009 no

## Preprocessing

In [46]:
target_features = ['item_count', 'order_count']
useless_features = ['total_revenue', 'avg_order_value', 'avg_items_per_order']
x, y = df.drop(target_features, axis=1), df[target_features]
x = x.drop(useless_features, axis=1)

# Drop datetime column (it's already encoded into temporal features)
x = x.drop('datetime', axis=1)

# Handle null values
print(f"Null values before handling:\n{x.isnull().sum()[x.isnull().sum() > 0]}\n")

# Strategy 1: Drop latitude/longitude (40% missing - too much to impute reliably)
x = x.drop(['longitude', 'latitude'], axis=1)

# Strategy 2: Fill remaining nulls with appropriate values
# For categorical: use mode or create 'unknown' category
x['type_id'] = x['type_id'].fillna(-1)  # -1 indicates missing type

# For numerical restaurant features: use median (robust to outliers)
x['waiting_time'] = x['waiting_time'].fillna(x['waiting_time'].median())
x['rating'] = x['rating'].fillna(x['rating'].median())
x['delivery'] = x['delivery'].fillna(0)  # Binary: assume no delivery if missing
x['accepting_orders'] = x['accepting_orders'].fillna(0)  # Binary: assume not accepting if missing

print(f"Null values after handling:\n{x.isnull().sum().sum()} total nulls remaining")

# Convert all object/string dtypes to avoid serialization issues with multiprocessing
x['place_id'] = x['place_id'].astype('float64')
x['type_id'] = x['type_id'].astype('float64')
x['is_holiday'] = x['is_holiday'].astype('int')

# Fix column index dtype issue - convert to regular Python strings
x.columns = x.columns.astype(str)
y.columns = y.columns.astype(str)

Null values before handling:
type_id             1578
waiting_time           2
rating                 2
delivery               2
accepting_orders       2
dtype: int64

Null values after handling:
0 total nulls remaining


In [47]:
# split the time series data into train and test sets
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [48]:
scale_features = [
    # Restaurant features (varying ranges)
    'waiting_time',           # Minutes (0-60+)
    'rating',                 # 0-5 scale
    'avg_discount',           # Percentage (0-100)
    
    # Lag/historical features (count data, large range)
    'prev_hour_items',        # Can be 0-1000+
    'prev_day_items',         # Can be 0-10000+
    'prev_week_items',        # Can be 0-50000+
    'prev_month_items',       # Can be 0-200000+
    'rolling_7d_avg_items',   # Average counts
    
    # Weather features (different units/scales)
    'temperature_2m',         # Celsius (-10 to 40+)
    'relative_humidity_2m',   # Percentage (0-100)
    'precipitation',          # mm (0-50+)
    'rain',                   # mm (0-50+)
    'snowfall',               # cm (0-50+)
    'cloud_cover',            # Percentage (0-100)
    'wind_speed_10m',         # km/h (0-100+)
    'weather_severity'        # Encoded scale
]

preprocessor = ColumnTransformer(
    transformers=[('sclaler', StandardScaler(), scale_features)],
    remainder='passthrough'
)

## Linear Regression (Baseline)

In [49]:
from sklearn.linear_model import LinearRegression

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(LinearRegression()))
])

linear_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

linear_model.fit(x_train, y_train)
y_train_pred_linear = linear_model.predict(x_train)
y_test_pred_linear = linear_model.predict(x_test)

# Evaluate on training set
print("\nTraining Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_train.iloc[:, i], y_train_pred_linear[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_train.iloc[:, i], y_train_pred_linear[:, i]):.4f}")
    print(f"  R2: {r2_score(y_train.iloc[:, i], y_train_pred_linear[:, i]):.4f}")

# Evaluate on test set
print("\nTest Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_test.iloc[:, i], y_test_pred_linear[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_test.iloc[:, i], y_test_pred_linear[:, i]):.4f}")
    print(f"  R2: {r2_score(y_test.iloc[:, i], y_test_pred_linear[:, i]):.4f}")


Training Set Performance:

item_count:
  RMSE: 446.0370
  MAE: 5.1902
  R2: -3.7767

order_count:
  RMSE: 50.4362
  MAE: 2.8187
  R2: -0.4884

Test Set Performance:

item_count:
  RMSE: 71.6137
  MAE: 4.4791
  R2: -0.0301

order_count:
  RMSE: 19.5026
  MAE: 2.6600
  R2: 0.1409


## Random Forests

### Initial Run (default hyperparameters)

In [50]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(RandomForestRegressor(n_jobs=-1, random_state=42)))
])

random_forest_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

random_forest_model.fit(x_train, y_train)
y_train_pred_rf = random_forest_model.predict(x_train)
y_test_pred_rf = random_forest_model.predict(x_test)

# Evaluate on training set
print("\nRandom Forest Training Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_train.iloc[:, i], y_train_pred_rf[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_train.iloc[:, i], y_train_pred_rf[:, i]):.4f}")
    print(f"  R2: {r2_score(y_train.iloc[:, i], y_train_pred_rf[:, i]):.4f}")

# Evaluate on test set
print("\nRandom Forest Test Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_test.iloc[:, i], y_test_pred_rf[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_test.iloc[:, i], y_test_pred_rf[:, i]):.4f}")
    print(f"  R2: {r2_score(y_test.iloc[:, i], y_test_pred_rf[:, i]):.4f}")


Random Forest Training Set Performance:

item_count:
  RMSE: 10.3632
  MAE: 1.6907
  R2: 0.8890

order_count:
  RMSE: 2.8046
  MAE: 0.8859
  R2: 0.9172

Random Forest Test Set Performance:

item_count:
  RMSE: 44.9750
  MAE: 4.1670
  R2: 0.3531

order_count:
  RMSE: 17.1261
  MAE: 2.6756
  R2: 0.2456


### Enhanced Run (tuned hyperparameters)

#### First Stage: structure + regularization

In [21]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(RandomForestRegressor(n_jobs=-1, random_state=42)))
])

random_forest_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

gs = GridSearchCV(
    estimator=random_forest_model,
    param_grid = {
        'regressor__model__estimator__max_depth': [10, 20],
        'regressor__model__estimator__min_samples_leaf': [2, 4],
        'regressor__model__estimator__max_features': ['sqrt', 0.5],
    },
    scoring='neg_mean_squared_error',
    cv=TimeSeriesSplit(n_splits=5),
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs.fit(x_train, y_train)
print(f"Best parameters found: {gs.best_params_}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters found: {'regressor__model__estimator__max_depth': 10, 'regressor__model__estimator__max_features': 0.5, 'regressor__model__estimator__min_samples_leaf': 4}

Random Forest Training Set Performance:

item_count:
  RMSE: 45.8082
  MAE: 4.1017
  R2: 0.5094

order_count:
  RMSE: 13.9589
  MAE: 2.2475
  R2: 0.5881

Random Forest Test Set Performance:

item_count:
  RMSE: 45.0272
  MAE: 4.1390
  R2: 0.3523

order_count:
  RMSE: 16.8069
  MAE: 2.6346
  R2: 0.2596


In [24]:
gs = GridSearchCV(
    estimator=random_forest_model,
    param_grid = {
        'regressor__model__estimator__max_depth': [5, 10, 15],
        'regressor__model__estimator__min_samples_leaf': [3, 4, 5],
        'regressor__model__estimator__max_features': [0.5, 'log2'],
    },
    scoring='neg_mean_squared_error',
    cv=TimeSeriesSplit(n_splits=5),
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs.fit(x_train, y_train)
print(f"Best parameters found: {gs.best_params_}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters found: {'regressor__model__estimator__max_depth': 15, 'regressor__model__estimator__max_features': 0.5, 'regressor__model__estimator__min_samples_leaf': 5}


In [25]:
gs = GridSearchCV(
    estimator=random_forest_model,
    param_grid = {
        'regressor__model__estimator__max_depth': [10, 15, 20, 25],
        'regressor__model__estimator__min_samples_leaf': [4, 5, 6, 7]
    },
    scoring='neg_mean_squared_error',
    cv=TimeSeriesSplit(n_splits=5),
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs.fit(x_train, y_train)
print(f"Best parameters found: {gs.best_params_}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters found: {'regressor__model__estimator__max_depth': 10, 'regressor__model__estimator__min_samples_leaf': 7}


#### Second: stability

In [23]:
gs2 = GridSearchCV(
    estimator=random_forest_model,
    param_grid = {
        'regressor__model__estimator__n_estimators': [100, 200, 400],
        'regressor__model__estimator__bootstrap': [True, False],
    },
    scoring='neg_mean_squared_error',
    cv=TimeSeriesSplit(n_splits=5),
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs2.fit(x_train, y_train)
print(f"Best parameters found: {gs2.best_params_}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters found: {'regressor__model__estimator__bootstrap': True, 'regressor__model__estimator__n_estimators': 400}


In [28]:
gs2 = GridSearchCV(
    estimator=random_forest_model,
    param_grid = {
        'regressor__model__estimator__n_estimators': [400, 500, 600],
        'regressor__model__estimator__bootstrap': [True, False],
    },
    scoring='neg_mean_squared_error',
    cv=TimeSeriesSplit(n_splits=5),
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs2.fit(x_train, y_train)
print(f"Best parameters found: {gs2.best_params_}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters found: {'regressor__model__estimator__bootstrap': True, 'regressor__model__estimator__n_estimators': 600}


### Best Model

In [51]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(RandomForestRegressor(
        n_jobs=-1,
        random_state=42,
        max_depth=12,
        min_samples_leaf=7,
        max_features=0.5,
        n_estimators=600,
        bootstrap=True
    )))
])

best_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

best_model.fit(x_train, y_train)

y_train_pred_best = best_model.predict(x_train)
y_test_pred_best = best_model.predict(x_test)

# Evaluate on training set
print("\nRandom Forest Training Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_train.iloc[:, i], y_train_pred_best[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_train.iloc[:, i], y_train_pred_best[:, i]):.4f}")
    print(f"  R2: {r2_score(y_train.iloc[:, i], y_train_pred_best[:, i]):.4f}")

# Evaluate on test set
print("\nRandom Forest Test Set Performance:")
for i, target in enumerate(target_features):
    print(f"\n{target}:")
    print(f"  RMSE: {mean_squared_error(y_test.iloc[:, i], y_test_pred_best[:, i]):.4f}")
    print(f"  MAE: {mean_absolute_error(y_test.iloc[:, i], y_test_pred_best[:, i]):.4f}")
    print(f"  R2: {r2_score(y_test.iloc[:, i], y_test_pred_best[:, i]):.4f}")


Random Forest Training Set Performance:

item_count:
  RMSE: 42.0605
  MAE: 3.8744
  R2: 0.5496

order_count:
  RMSE: 12.8727
  MAE: 2.1199
  R2: 0.6201

Random Forest Test Set Performance:

item_count:
  RMSE: 43.9296
  MAE: 4.1022
  R2: 0.3681

order_count:
  RMSE: 16.7582
  MAE: 2.6258
  R2: 0.2618


## Saving the Model

In [52]:
metadata = {
    'python_version': sys.version,
    'sklearn_version': sys.modules['sklearn'].__version__,
    'numpy_version': sys.modules['numpy'].__version__,
    'model_type': 'RandomForestRegressor',
    'preprocessing': {
        'scaled_features': scale_features,
        'imputation_strategies': {
            'type_id': 'mode (-1 for unknown)',
            'waiting_time': 'median',
            'rating': 'median',
            'delivery': 'fill 0',
            'accepting_orders': 'fill 0'
        }
    },
    'hyperparameters': {
        'max_depth': 12,
        'min_samples_leaf': 7,
        'max_features': 0.5,
        'n_estimators': 600,
        'bootstrap': True
    },
    'training_size': len(x_train),
    'test_size': len(x_test)
}

joblib.dump(best_model, '../data/models/rf_model.joblib')
joblib.dump(metadata, '../data/models/rf_model_metadata.json')

['../data/models/rf_model_metadata.json']

## DONE!